# Setup

In [1]:
import json
import requests
from datetime import datetime as dt
from collections import defaultdict
import pandas as pd

In [2]:
#https://pypi.org/project/gamma-viewer/
from gamma_viewer import GammaViewer
from IPython.display import display, Markdown

In [3]:
def printjson(j):
    print(json.dumps(j,indent=4))
def print_json(j):
    printjson(j)

In [21]:
def post(name,url,message,params=None):
    if params is None:
        response = requests.post(url,json=message)
    else:
        response = requests.post(url,json=message,params=params)
    if not response.status_code == 200:
        print(name, 'error:',response.status_code)
        print(response.json())
        return {}
    return response.json()

def automat(db,message):
    automat_url = f'https://automat.renci.org/{db}/query'
    response = requests.post(automat_url,json=message)
    print(response.status_code)
    return response.json()

def strider(message):
    url = 'https://strider.renci.org/1.1/query?log_level=DEBUG'
    strider_answer = post(strider,url,message)
    return strider_answer

def aragorn(message, coalesce_type='xnone'):
    if coalesce_type == 'xnone':
        answer = post('aragorn','https://aragorn.renci.org/1.1/query',message)
    else:
        answer = post('aragorn','https://aragorn.renci.org/1.1/query',message, params={'answer_coalesce_type':coalesce_type})
    return answer

##

def bte(message):
    url = 'https://api.bte.ncats.io/v1/query'
    return post(strider,url,message)

def coalesce(message,method='all'):
    url = 'https://answercoalesce.renci.org/coalesce/graph'
    return post('AC'+method,url,message)

def striderandfriends(message):
    strider_answer = strider(message)    
    coalesced_answer = post('coalesce','https://answercoalesce.renci.org/coalesce/all',strider_answer)
    omni_answer = post('omnicorp','https://aragorn-ranker.renci.org/omnicorp_overlay',coalesced_answer)
    weighted_answer = post('weight','https://aragorn-ranker.renci.org/weight_correctness',omni_answer)
    scored_answer = post('score','https://aragorn-ranker.renci.org/score',weighted_answer)
    return strider_answer,coalesced_answer,omni_answer,weighted_answer,scored_answer

In [5]:
def print_errors(strider_result):
    errorcounts = defaultdict(int)
    for logmessage in strider_result['logs']:
        if logmessage['level'] == 'ERROR':
            jm = json.loads(logmessage['message'])
            words = jm['error'].split()
            e = " ".join(words[:-5])
            errorcounts[e] += 1
    for error,count in errorcounts.items():
        print(f'{error} ({count} times)')
        
def print_queried_sources(strider_result):
    querycounts = defaultdict(int)
    for logmessage in strider_result['logs']:
        if 'step' in logmessage and isinstance(logmessage['step'],list):
            for s in logmessage['step']:
                querycounts[s['url']] += 1
    for url,count in querycounts.items():
        print(f'{url} ({count} times)')
        
def print_query_for_source(strider_result,url):
    for logmessage in strider_result['logs']:
        if 'step' in logmessage and isinstance(logmessage['step'],list):
            for s in logmessage['step']:
                if s['url']==url:
                    print(s)

In [6]:
def retrieve_ars_results(mid):
    message_url = f'https://ars.transltr.io/ars/api/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    results = {}
    for child in j['children']:
        if child['actor']['agent'] in ['ara-aragorn', 'ara-aragorn-exp']:
            childmessage_id = child['message']
            child_url = f'https://ars.transltr.io/ars/api/messages/{childmessage_id}'
            child_response = requests.get(child_url).json()
            try:
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except:
                nresults=0
            print( child['status'], child['actor']['agent'],nresults )
    return results

In [7]:
def get_provenance(message):
    """Given a message with results, find the source of the edges"""
    prov = defaultdict(lambda: defaultdict(int)) # {qedge->{source->count}}
    results = message['message']['results']
    kg = message['message']['knowledge_graph']['edges']
    edge_bindings = [ r['edge_bindings'] for r in results ]
    for bindings in edge_bindings:
        for qg_e, kg_l in bindings.items():
            for kg_e in kg_l:
                for att in kg[kg_e['id']]['attributes']:
                    if att['name'] == 'provenance':
                        source = att['value']
                        prov[qg_e][source]+=1
    qg_edges = []
    sources = []
    counts = []
    for qg_e in prov:
        for source in prov[qg_e]:
            qg_edges.append(qg_e)
            sources.append(source)
            counts.append(prov[qg_e][source])
    prov_table = pd.DataFrame({"QG Edge":qg_edges, "Source":sources, "Count":counts})
    return prov_table

## Query Specific

In [8]:
standup_json='StandupDefinitions/standup_19.json'

In [15]:
with open(standup_json,'r') as jsonfile:
    standup_info = json.load(jsonfile)

In [16]:
display(Markdown(f"# {standup_info['Query Title']}"))
display(Markdown(f"{standup_info['Query Description']}"))
print(f'Github Issue: {standup_info["github_issue"]}')

# 19. RHOBTB2 Step 1

Genes that downregulate RHOBTB2 (NCBIGene:23221)

Github Issue: https://github.com/NCATSTranslator/testing/issues/68


The query as run through the ARS:

In [17]:
query = json.loads(requests.get(standup_info['query_location']).content)
printjson(query)

{
    "message": {
        "query_graph": {
            "edges": {
                "e01": {
                    "object": "n0",
                    "subject": "n1",
                    "predicates": [
                        "biolink:negatively_regulates_entity_to_entity"
                    ]
                }
            },
            "nodes": {
                "n0": {
                    "ids": [
                        "NCBIGene:23221"
                    ],
                    "categories": [
                        "biolink:Gene"
                    ]
                },
                "n1": {
                    "categories": [
                        "biolink:Gene"
                    ]
                }
            }
        }
    }
}


## ARS Assessment

In [18]:
ARS_Responses = [(dt.strptime(x['ARS_result_date'],'%Y-%m-%d'),x['ARS_result_id']) for x in standup_info['ARS_Results']]
ARS_Responses.sort()

In [19]:
for ars_date, ars_id in ARS_Responses:
    display(Markdown(f'### {ars_date}'))
    _ = retrieve_ars_results(ars_id)
    print(f'https://arax.ncats.io/?source=ARS&id={ars_id}')

### 2021-05-27 00:00:00

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [37]:
query['message']['query_graph']['edges']['e01']['predicates'][0]='biolink:related_to'

In [38]:
printjson(query)

{
    "message": {
        "query_graph": {
            "edges": {
                "e01": {
                    "object": "n0",
                    "subject": "n1",
                    "predicates": [
                        "biolink:related_to"
                    ]
                }
            },
            "nodes": {
                "n0": {
                    "ids": [
                        "NCBIGene:23221"
                    ],
                    "categories": [
                        "biolink:Gene"
                    ]
                },
                "n1": {
                    "categories": [
                        "biolink:Gene"
                    ]
                }
            }
        }
    }
}


## Strider Direct

In [39]:
start = dt.now()
strider_result = strider(query)
end = dt.now()
print(f"Strider produced {len(strider_result['message']['results'])} results in {end-start}.")

Strider produced 14 results in 0:00:36.436705.


### Provenance

In [40]:
prov = get_provenance(strider_result)
display(prov)

KeyError: 'name'

### Queried sources

In [41]:
print_queried_sources(strider_result)

### Errors

In [42]:
print_errors(strider_result)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Results

In [43]:
view = GammaViewer(props={"data":strider_result})
display(view)

### Strider Assessment

Enter Assessment Here

## ARAGORN 

In [23]:
start = dt.now()
aragorn_result = aragorn(query)
end = dt.now()
if 'results' in aragorn_result['message']:
    print(f"ARAGORN produced {len(aragorn_result['message']['results'])} results in {end-start}.")
else:
    print('Error, no result field')

Error, no result field


In [28]:
print_json(aragorn_result['message'])

{
    "query_graph": {
        "nodes": {
            "n0": {
                "ids": [
                    "NCBIGene:23221"
                ],
                "categories": [
                    "biolink:Gene"
                ],
                "is_set": false
            },
            "n1": {
                "categories": [
                    "biolink:Gene"
                ],
                "is_set": false
            }
        },
        "edges": {
            "e01": {
                "subject": "n1",
                "object": "n0",
                "predicates": [
                    "biolink:negatively_regulates_entity_to_entity"
                ]
            }
        }
    }
}


In [20]:
view = GammaViewer(props={"data":aragorn_result})
display(view)

In [27]:
tq={
    "message": {
        "query_graph": {
            "nodes": {
                "n1": {
                    "id": "HP:0033127",
                    "category": "biolink:PhenotypicFeature"
                },
                "n2": {
                    "category": "biolink:Disease"
                }
            },
            "edges": {
                "e02": {
                    "subject": "n1",
                    "object": "n2",
                    "predicate": "biolink:affected_by"
                }
            }
        }
    }
}

In [29]:
r = automat('uberongraph',tq)

200


### ARAGORN Assessment

How did we do?

In [45]:
query = requests.get('https://kp-registry.renci.org/kps').json()
printjson(query)

{
    "Genetics KP": "https://translator.broadinstitute.org/genetics_provider/trapi/v1.1/query",
    "RTX KG2": "https://arax.ncats.io/api/rtxkg2/v1.1/query",
    "Automat Textmining KP (trapi v-1.1.0)": "https://automat.renci.org/textminingkp/1.1/query",
    "SPOKE KP": "https://spokekp.healthdatascience.cloud/api/v1.1//query",
    "CAM-KP API": "https://cam-kp-api-dev.renci.org/query",
    "Molecular Data Provider for NCATS Biomedical Translator Reasoners": "https://translator.broadinstitute.org/molepro/trapi/v1.1/query",
    "Automat Panther (trapi v-1.1.0)": "https://automat.renci.org/panther/1.1/query",
    "Automat MyChem (trapi v-1.1.0)": "https://automat.renci.org/mychem/1.1/query",
    "Automat KEGG (trapi v-1.1.0)": "https://automat.renci.org/kegg/1.1/query",
    "Automat Gtopdb (trapi v-1.1.0)": "https://automat.renci.org/gtopdb/1.1/query",
    "Automat IntAct (trapi v-1.1.0)": "https://automat.renci.org/intact/1.1/query",
    "Automat HGNC (trapi v-1.1.0)": "https://automat